Beat-n game with 10 pt limit

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates
using JLD2

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

const MXV = 9
const MNV = 1

function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end

function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get spuds with cost 10 and filter nondominated

In [3]:
function get_library(cost, n_init = 10000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff = Spud("", h,f,l,p,r,s,999)
                                ff_i += 1
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

function filter_nondominated2(as::Array{Spud})::Array{Spud}
    mat = get_payoffs(as);
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:length(as)
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

function spud2int(a::Spud)::Int64
    return a.h * 100000 + a.f * 10000 + a.l * 1000 + a.p * 100 + a.r * 10 + a.s
end

function eval_beatn(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    if length(bs)==0 || length(as)==0
        return 0
    end
    a = as[end]
    flag = true
    count = 0
    while flag
        if payoffs[a, bs[end-count]]==1
            count += 1
        else
            flag = false
        end
        if count == length(bs)
            flag = false
        end
    end
    return count
end

# tries all possible moves as extension of as
function try_beatn(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    ans = Array{Int64}(undef, size(payoffs)[1])
    for i in 1:size(payoffs)[1]
        as2 = copy(as)
        append!(as2, i)
        ans[i] = eval_beatn(payoffs, as2, bs)
    end
    return ans
end

# returns 1 if Player A lost
function lose_or_not(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    if maximum(scores) < prev_score
        return 1
    end
    return 0
end

function legal_moves(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .>= prev_score)
    return cands
end

# picks randomly
function choose_random(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .>= prev_score)
    return rand(cands)
end

# picks aggressively
function choose_aggro(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .== maximum(scores))
    return rand(cands)
end

function play_random_vs_aggro(
        payoffs::Array{Int64}, 
        as::Array{Int64} = zeros(Int64, 0), 
        bs::Array{Int64} = zeros(Int64, 0))::Tuple{Int64, Array{Int64}, Array{Int64}}
    if length(as) == 0
        a = rand(1:size(payoffs)[1])
        as = append!(as, a)
    end
    if length(bs) < length(as)
        b = choose_aggro(payoffs, bs, as)
        bs = append!(bs, b)
    end
    flag = true
    a_win = 0
    while flag
        if lose_or_not(payoffs, as, bs)==1
            flag = false
            a_win = -1
        else
            # player A turn
            a = choose_random(payoffs, as, bs)
            as = append!(as, a)
        end
        if flag
            if lose_or_not(payoffs, bs, as)==1
                flag = false
                a_win = 1
            else
                # player B turn
                b = choose_aggro(payoffs, bs, as)
                bs = append!(bs, b)
            end
        end
    end
    return (a_win, as, bs)
end

function play_random_vs_random(
        payoffs::Array{Int64}, 
        as::Array{Int64} = zeros(Int64, 0), 
        bs::Array{Int64} = zeros(Int64, 0))::Tuple{Int64, Array{Int64}, Array{Int64}}
    if length(as) == 0
        a = rand(1:size(payoffs)[1])
        as = append!(as, a)
    end
    if length(bs) < length(as)
        b = choose_aggro(payoffs, bs, as)
        bs = append!(bs, b)
    end
    flag = true
    a_win = 0
    while flag
        if lose_or_not(payoffs, as, bs)==1
            flag = false
            a_win = -1
        else
            # player A turn
            a = choose_random(payoffs, as, bs)
            as = append!(as, a)
        end
        if flag
            if lose_or_not(payoffs, bs, as)==1
                flag = false
                a_win = 1
            else
                # player B turn
                b = choose_random(payoffs, bs, as)
                bs = append!(bs, b)
            end
        end
    end
    return (a_win, as, bs)
end

function display_game(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})
    for i in 1:length(as)
        print(libi[as[i]])
        print(" {")
        n = eval_beatn(payoffs, as[1:i], bs[1:(i-1)])
        print(n)
        print("}. ")
        if i <= length(bs)
            print(libi[bs[i]])
            print(" {")
            n = eval_beatn(payoffs, bs[1:i], as[1:i])
            print(n)
            print("}. ")
        end
        println()
    end
end


lib0 = get_library(10);
lib=filter_nondominated2(lib0);
payoffs = get_payoffs(lib);
s2ind = Dict(lib[i] => i for i in 1:length(lib));
int2ind = Dict(spud2int(lib[i]) => i for i in 1:length(lib));
libi = [spud2int(ff) for ff in lib];
nlib = length(lib)

74

In [4]:
for ff in lib
    print(" ")
    print(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
end


 111124 111133 111142 111214 111223 111232 111241 111313 111322 111331 111412 111421 111511 112114 112123 112132 112141 112213 112222 112231 112312 112321 112411 113113 113122 113131 113212 113221 113311 114112 114121 114211 121114 121123 121132 121213 121222 121231 121312 121321 121411 122113 122122 122212 122221 122311 123112 123211 131113 131122 131212 131221 131311 132112 132211 141112 141211 212113 212122 212131 212212 212221 212311 213112 213121 213211 214111 222112 222211 312112 312121 312211 313111 412111

In [5]:
# check game lengths
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a3 = int2ind[121321]
b3 = int2ind[121222]
a4 = int2ind[113122]
b4 = int2ind[111133]
#a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a3,a4],[b1,b2,b3,b4])
a_win, as, bs = play_random_vs_random(payoffs, [a1,a2,a3,a4],[b1,b2,b3,b4])
display_game(payoffs, as, bs)

121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
113122 {0}. 111133 {2}. 
313111 {2}. 113131 {2}. 
214111 {2}. 114112 {3}. 
112213 {4}. 


## MCTS

In [6]:
function intarr_to_string(as::Array{Int64})::String
    return join([convert(Char, a+32) for a in as])
end

function string_to_intarr(s::String)::Array{Int64}
    return [convert(Int64, s[i])-32 for i in 1:length(s)]
end

# s=intarr_to_string([i for i in 1:75])
# as=string_to_intarr(s)
# for i in as
#     print(i)
#     print(" ")
# end

function split_arrs(zs::Array{Int64})::Tuple{Array{Int64},Array{Int64}}
    if mod(length(zs), 2)==0
        return (zs[1:2:(end-1)], zs[2:2:end])
    else
        return (zs[1:2:end], zs[2:2:(end-1)])
    end
end

function merge_arrs(as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    zs = Array{Int64}(undef, length(as)+length(bs))
    if mod(length(zs), 2)==0
        zs[1:2:(end-1)]=as
        zs[2:2:end]=bs
    else
        zs[1:2:end] = as
    zs[2:2:(end-1)] = bs
    end
    return zs
end

function propagate_win(tree::Dict{String, Vector{Int64}}, leaf::String, win::Int64)::Dict{String, Vector{Int64}}
    for i in 0:length(leaf)
        if i==0
            node=""
        else
            node=leaf[1:i]
        end
        if haskey(tree, node)
            tree[node][2] += 1
            tree[node][1] += win
        end
    end
    return tree
end

function append_move_to_leaf(leaf::String, move::Int64)::String
    zs = string_to_intarr(leaf)
    append!(zs, move)
    return intarr_to_string(zs)
end

function get_children(payoffs::Array{Int64}, leaf::String)::Array{String}
    # get arrays
    as, bs = split_arrs(string_to_intarr(leaf))
    if mod(length(leaf), 2)==0
        player = 1
    else
        player = 2
    end
    # generate successors
    if player==1
        cands = legal_moves(payoffs, as, bs)
    else
        cands = legal_moves(payoffs, bs, as)
    end
    return [append_move_to_leaf(leaf, c) for c in cands]
end

function check_and_propagate_terminal(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)::Dict{String, Vector{Int64}}
    children = get_children(payoffs, leaf)
    if length(children) == 0
        # indicate terminal
        tree[leaf][3]=1
        if mod(length(leaf), 2)==0
            #p1 lost
            tree = propagate_win(tree, leaf, 0)
        else
            #p1 won
            tree = propagate_win(tree, leaf, 1)
        end
    end
    return tree
end

function try_add_node(tree::Dict{String, Vector{Int64}}, node::String)::Dict{String, Vector{Int64}}
    if haskey(tree, node)
        return tree
    else
        tree[node] = [0,0,0]
        return tree
    end
end

function query_node(tree, node)::Array{Int64}
    if haskey(tree, node)
        return tree[node]
    else
        return [0,0,0]
    end
end

function expand_leaf(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)::Dict{String, Vector{Int64}}
    # check if leaf is terminal
    tree = check_and_propagate_terminal(payoffs, tree, leaf)
    if tree[leaf][3]==1
        return tree
    end
    # get arrays for leaf
    as, bs = split_arrs(string_to_intarr(leaf))
    if mod(length(leaf), 2)==0
        player = 1
    else
        player = 2
    end
    # generate random successor
    if player==1
        cands = legal_moves(payoffs, as, bs)
        newleaf = append_move_to_leaf(leaf, rand(cands))
    else
        cands = legal_moves(payoffs, bs, as)
        newleaf = append_move_to_leaf(leaf, rand(cands))
    end
    tree = try_add_node(tree, newleaf)
    tree = check_and_propagate_terminal(payoffs, tree, newleaf)
    # play the match
    as, bs = split_arrs(string_to_intarr(newleaf))
    a_win, as2, bs2 = play_random_vs_random(payoffs, as, bs)
    # propagate result
    if a_win == 1
        tree = propagate_win(tree, newleaf, 1)
    else
        tree = propagate_win(tree, newleaf, 0)
    end
    return tree
end

function sample_next_node(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String, 
        explore_const::Float64=1.4)::String
    # values for UCT
    n = tree[node][2]
    # assumes that all children are added
    children = get_children(payoffs, node)
    counts = [query_node(tree, c)[2] for c in children]
    wins = [query_node(tree, c)[1] for c in children]
    if minimum(counts) == 0
        sel = children[rand(findall(counts .== 0))]
        return sel
    end
    if mod(length(node), 2)==1
        wins = counts .- wins
    end
    scores = counts .* 0.0
    for i in 1:length(scores)
        if counts[i] > 0
            scores[i] = wins[i]/counts[i] + explore_const * sqrt((log(n)/counts[i]))
        end
    end
    sel = children[rand(findall(scores .== maximum(scores)))]
    return sel
end
                            
function random_leaf(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String="")::String
    while query_node(tree,node)[2] > 1 && query_node(tree,node)[3] != 1
        node = sample_next_node(payoffs, tree, node)
    end
    return node
end
                                    
# exploit mode
function sample_next_node2(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String, beta::Float64 = 1.0)::String
    # values for UCT
    n = tree[node][2]
    # assumes that all children are added
    children = get_children(payoffs, node)
    counts = [query_node(tree, c)[2] for c in children]
    wins = [query_node(tree, c)[1] for c in children]
    if mod(length(node), 2)==1
        wins = counts .- wins
    end
    scores = counts .* 0.0
    for i in 1:length(scores)
        if counts[i] > 0
            scores[i] = wins[i]/counts[i]
        end
    end
    scores = [exp(beta*x) for x in scores]
    scores = scores./sum(scores)
    cs = cumsum(scores)
    z = rand(1)[1]
    sel = children[minimum(findall(cs .>= z))]
    return sel
end

function display_node(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String)
    print("Node: ")
    println(node)
    as, bs = split_arrs(string_to_intarr(node))
    display_game(payoffs, as, bs)
    print("Stats: ")
    print(query_node(tree,node)[1])
    print("/")
    print(query_node(tree,node)[2])
    print("=")
    s = string(query_node(tree,node)[1]/query_node(tree,node)[2])
    println(s[1:min(5, length(s))])
    if query_node(tree,node)[3]==1
        println("[terminal] ")
    end
end

function display_path(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)
    for i in 0:length(leaf)
        if i==0
            if haskey(tree, "")
                display_node(payoffs, tree, "")
                println()
            end
        else
            if haskey(tree, leaf[1:i])
                display_node(payoffs, tree, leaf[1:i])
                println()
            end
        end
    end
end

function display_choices(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String)
    children = get_children(payoffs, node)
    display_node(payoffs, tree, node)
    for c in children
        println()
        display_node(payoffs, tree, c)
    end
end

function create_tree(node::String = "")::Dict{String, Vector{Int64}}
    return Dict(node => [0,0,0])
end

function grow_tree(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String = "")::Dict{String, Vector{Int64}}
    node = random_leaf(payoffs, tree, node)
    tree = try_add_node(tree, node)
    tree = check_and_propagate_terminal(payoffs, tree, node)
    tree = expand_leaf(payoffs, tree, node)
    return tree
end

function grow_tree(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}; 
        node::String = "", iters::Int64 = 1000)::Dict{String, Vector{Int64}}
    for i in 1:iters
        tree = grow_tree(payoffs, tree, node)
    end
    return tree
end

grow_tree (generic function with 2 methods)

## Play vs MCTS
### CPU first

In [128]:
n_mc = 100000
beta = 20.0
#game = "c9\\"
game = ""
tree = create_tree(game)
display_node(payoffs, create_tree(game), game)

Node: 
Stats: 0/0=NaN


In [161]:
# player turn
pmove = int2ind[312121]
game = append_move_to_leaf(game, pmove)
display_node(payoffs, create_tree(game), game)

Node: GbEH="dg
121312 {0}. 213211 {0}. 
121222 {1}. 121321 {2}. 
113311 {2}. 111133 {3}. 
222112 {3}. 312121 {4}. 
Stats: 0/0=NaN


In [158]:
# MCTS turn
tree = create_tree(game);
@time tree = grow_tree(payoffs, tree, node=game, iters=n_mc);
game = sample_next_node2(payoffs, tree, game, beta);
display_node(payoffs, create_tree(game), game)

  9.284893 seconds (109.55 M allocations: 9.434 GiB, 32.29% gc time)
Node: GbEH="d
121312 {0}. 213211 {0}. 
121222 {1}. 121321 {2}. 
113311 {2}. 111133 {3}. 
222112 {3}. 
Stats: 0/0=NaN


In [162]:
# cheat
display_node(payoffs, tree, game)

Node: GbEH="dg
121312 {0}. 213211 {0}. 
121222 {1}. 121321 {2}. 
113311 {2}. 111133 {3}. 
222112 {3}. 312121 {4}. 
Stats: 0/87388=0.0
[terminal] 


In [117]:
# cheat and consult MCTS for next move
tree = create_tree(game);
@time tree = grow_tree(payoffs, tree, node=game, iters=n_mc);

 14.219853 seconds (164.53 M allocations: 15.641 GiB, 32.10% gc time)


In [160]:
move = sample_next_node2(payoffs, tree, game, 1.0);
display_node(payoffs, tree, move)
move = sample_next_node2(payoffs, tree, game, 5.0);
display_node(payoffs, tree, move)
move = sample_next_node2(payoffs, tree, game, 5.0);
display_node(payoffs, tree, move)
move = sample_next_node2(payoffs, tree, game, 20.0);
display_node(payoffs, tree, move)
move = sample_next_node2(payoffs, tree, game, 20.0);
display_node(payoffs, tree, move)


Node: GbEH="d3
121312 {0}. 213211 {0}. 
121222 {1}. 121321 {2}. 
113311 {2}. 111133 {3}. 
222112 {3}. 112222 {3}. 
Stats: 61/176=0.346
Node: GbEH="dG
121312 {0}. 213211 {0}. 
121222 {1}. 121321 {2}. 
113311 {2}. 111133 {3}. 
222112 {3}. 121312 {3}. 
Stats: 343/15364=0.022
Node: GbEH="dG
121312 {0}. 213211 {0}. 
121222 {1}. 121321 {2}. 
113311 {2}. 111133 {3}. 
222112 {3}. 121312 {3}. 
Stats: 343/15364=0.022
Node: GbEH="dG
121312 {0}. 213211 {0}. 
121222 {1}. 121321 {2}. 
113311 {2}. 111133 {3}. 
222112 {3}. 121312 {3}. 
Stats: 343/15364=0.022
Node: GbEH="dg
121312 {0}. 213211 {0}. 
121222 {1}. 121321 {2}. 
113311 {2}. 111133 {3}. 
222112 {3}. 312121 {4}. 
Stats: 0/87388=0.0
[terminal] 


In [156]:
move = sample_next_node2(payoffs, tree, game, 0.0);
display_node(payoffs, tree, move)
move = sample_next_node2(payoffs, tree, game, 0.0);
display_node(payoffs, tree, move)
move = sample_next_node2(payoffs, tree, game, 0.0);
display_node(payoffs, tree, move)
move = sample_next_node2(payoffs, tree, game, 0.0);
display_node(payoffs, tree, move)
move = sample_next_node2(payoffs, tree, game, 0.0);
display_node(payoffs, tree, move)

Node: GbEH=L
121312 {0}. 213211 {0}. 
121222 {1}. 121321 {2}. 
113311 {2}. 122212 {2}. 
Stats: 36/80=0.45
Node: GbEH="
121312 {0}. 213211 {0}. 
121222 {1}. 121321 {2}. 
113311 {2}. 111133 {3}. 
Stats: 111/1743=0.063
Node: GbEH=+
121312 {0}. 213211 {0}. 
121222 {1}. 121321 {2}. 
113311 {2}. 111412 {3}. 
Stats: 114/3290=0.034
Node: GbEH=g
121312 {0}. 213211 {0}. 
121222 {1}. 121321 {2}. 
113311 {2}. 312121 {3}. 
Stats: 92/5765=0.015
Node: GbEH="
121312 {0}. 213211 {0}. 
121222 {1}. 121321 {2}. 
113311 {2}. 111133 {3}. 
Stats: 111/1743=0.063


In [10]:
length(get_children(payoffs, game))==0

false

In [11]:
# get_children(payoffs, game)

In [12]:
node = game[1:2]
display_node(payoffs, create_tree(node), node)

Node: c9
214111 {0}. 113122 {0}. 
Stats: 0/0=NaN


In [13]:
tree = create_tree(node)
@time tree = grow_tree(payoffs, tree, node=node, iters=n_mc)

  2.919289 seconds (46.16 M allocations: 3.648 GiB, 16.33% gc time)


Dict{String, Vector{Int64}} with 17405 entries:
  "c92aS!"   => [0, 1, 0]
  "c9\"aEF"  => [0, 1, 0]
  "c9+-K"    => [1, 1, 0]
  "c9`f<"    => [0, 1, 0]
  "c9EhP"    => [0, 1, 0]
  "c9ibQZ"   => [0, 1, 0]
  "c9\"#'"   => [0, 1, 0]
  "c9?@%"    => [0, 1, 0]
  "c9+hW"    => [1, 1, 0]
  "c9S*>]"   => [0, 1, 0]
  "c9\\BW"   => [0, 1, 0]
  "c9ifd"    => [1, 1, 0]
  "c9i9(="   => [1, 1, 0]
  "c9F*+"    => [1, 1, 0]
  "c9:eE5"   => [1, 1, 0]
  "c9YXJ"    => [0, 1, 0]
  "c9f4i;"   => [1, 1, 0]
  "c9iR:"    => [1, 1, 0]
  "c9:&3"    => [0, 1, 0]
  "c9i'C"    => [1, 1, 0]
  "c9H7K4"   => [1, 1, 0]
  "c9_\$"    => [0, 1, 0]
  "c9'fe"    => [0, 1, 0]
  "c9f\\_"   => [0, 1, 0]
  "c9\"4>\$" => [1, 1, 0]
  ⋮          => ⋮

## Mate in X analysis

In [14]:
function mate_checker(
        payoffs::Array{Int64},  
        node00::String;
        nits_tree::Int64 = 3,
        nits_cands::Int64 = 1000,
        verbose::Bool = true
        )::Tuple{Float64, Int64}
    tree = create_tree(node00)
    for i in 1:nits_tree
        tree = expand_game_tree(payoffs, tree)
    end
    # Check mate-in-X property
    lens = [length(k) for k in keys(tree)]
    minlen = minimum(lens)
    maxlen = maximum(lens)
    game_values = Dict(node00 => 0.0)
    for len in maxlen:-1:minlen
        wvalue = -1.0 # if len is even and node is terminal, then pl 1 loses
        if mod(len, 2)==1
            wvalue = 1.0
        end
        for node in [nd for nd in keys(tree) if length(nd)==len]
            if tree[node][3]==1
                game_values[node]=wvalue
            elseif length(node)==maxlen
                game_values[node]=0
            else
                children=get_children(payoffs, node)
                vals = [game_values[c] for c in children]
                if mod(len, 2)==1
                    # player 2 to move: get minimum of children values
                    game_values[node] = minimum(vals)
                else
                    # player 1 to move: get maximum of children values
                    game_values[node] = maximum(vals)
                end
            end
        end
    end
    # check length of best resistance
    best_res_length = Dict{String,Int64}()
    val = game_values[node00]
    if val > 0
        resister = 1
    else
        resister = 0 # player 2
    end
    for iter in 1:nits_cands
        node = node00
        val = game_values[node00]
        flag = true
        while flag
            cands = [c for c in keys(game_values) if length(c)==(length(node)+1) && c[1:length(node)]==node && game_values[c]==val]
            if length(cands) > 0
                node = rand(cands)
            else
                flag = false
            end
        end
        best_res_length[node] = length(node)-minlen
    end
    # Range of BRs
    lens_br = [length(k) for k in keys(best_res_length)]
    minlen_br = minlen
    maxlen_br = maximum(lens_br)
    for len in maxlen_br:-1:minlen_br
        nodes = unique([node[1:len] for node in keys(best_res_length) if length(node) >= len])
        for node in nodes
            if haskey(best_res_length, node)
                # do nothing
            else
                children = [c for c in keys(best_res_length) if length(c) == (length(node)+1) && c[1:length(node)]==node]
                brs = [best_res_length[c] for c in children]
                if mod(length(node), 2)==resister
                    # current node is resister to play
                    best_res_length[node] = maximum(brs)
                else
                    best_res_length[node] = minimum(brs)                
                end
            end
        end
    end
    if verbose
        node = node00
        br = best_res_length[node00]
        flag = true
        while flag
            cands = [nd for nd in keys(best_res_length) if length(nd)==(length(node)+1) && best_res_length[nd]==br && nd[1:length(node)]==node]
            if length(cands)==0
                flag = false
            else
                node = rand(cands)
            end
        end
        display_path(payoffs, tree, node)
    end
    return (game_values[node00], best_res_length[node00])
end

function expand_game_tree(payoffs::Array{Int64},  tree::Dict{String, Vector{Int64}})::Dict{String, Vector{Int64}}
    maxlen = maximum([length(nd) for nd in keys(tree)])
    nodes = [nd for nd in keys(tree) if length(nd)==maxlen]
    for node in nodes
        children = get_children(payoffs, node)
        for c in children
            tree = try_add_node(tree, c)
            tree = check_and_propagate_terminal(payoffs, tree, c)
        end
    end
    return tree
end

expand_game_tree (generic function with 1 method)

In [15]:
node00 = "%ZU7C`:#"
@time mate_checker(payoffs, node00, nits_tree=4, verbose=true)

Node: %ZU7C`:#
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
Stats: 81/228=0.355

Node: %ZU7C`:#a
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
213121 {4}. 
Stats: 2/2=1.0

Node: %ZU7C`:#aE
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
213121 {4}. 121222 {5}. 
Stats: 2/2=1.0

Node: %ZU7C`:#aE;
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
213121 {4}. 121222 {5}. 
113212 {5}. 
Stats: 1/1=1.0
[terminal] 

  1.307653 seconds (6.72 M allocations: 242.064 MiB, 3.87% gc time, 54.66% compilation time)


(1.0, 3)

#### debugging code

In [16]:
node00 = "%ZU7C`:#"
tree = create_tree(node00)
for i in 1:3
    tree = expand_game_tree(payoffs, tree)
end
display_node(payoffs, tree, node00)

Node: %ZU7C`:#
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
Stats: 81/82=0.987


In [17]:
[nd for nd in keys(tree) if tree[nd][3]==1 && length(nd)==11]

81-element Vector{String}:
 "%ZU7C`:#E2c"
 "%ZU7C`:#E&a"
 "%ZU7C`:#;+c"
 "%ZU7C`:#E38"
 "%ZU7C`:#E3c"
 "%ZU7C`:#E&8"
 "%ZU7C`:#L+a"
 "%ZU7C`:#;5c"
 "%ZU7C`:#G+c"
 "%ZU7C`:#aE;"
 "%ZU7C`:#E5a"
 "%ZU7C`:#L)c"
 "%ZU7C`:#;%c"
 ⋮
 "%ZU7C`:#E;c"
 "%ZU7C`:#G+a"
 "%ZU7C`:#;)a"
 "%ZU7C`:#G5b"
 "%ZU7C`:#G)c"
 "%ZU7C`:#G%a"
 "%ZU7C`:#E+b"
 "%ZU7C`:#E5b"
 "%ZU7C`:#L3c"
 "%ZU7C`:#E58"
 "%ZU7C`:#E]c"
 "%ZU7C`:#G)b"

In [18]:
display_path(payoffs, tree, "%ZU7C`:#G5b")

Node: %ZU7C`:#
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
Stats: 81/82=0.987

Node: %ZU7C`:#G
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
121312 {3}. 
Stats: 12/12=1.0

Node: %ZU7C`:#G5
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
121312 {3}. 112312 {4}. 
Stats: 3/3=1.0

Node: %ZU7C`:#G5b
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
121312 {3}. 112312 {4}. 
213211 {5}. 
Stats: 1/1=1.0
[terminal] 



In [19]:
# Check mate-in-X property
lens = [length(k) for k in keys(tree)]
minlen = minimum(lens)
maxlen = maximum(lens)
game_values = Dict(node00 => 0.0)
for len in maxlen:-1:minlen
    wvalue = -1.0 # if len is even and node is terminal, then pl 1 loses
    if mod(len, 2)==1
        wvalue = 1.0
    end
    for node in [nd for nd in keys(tree) if length(nd)==len]
        if tree[node][3]==1
            game_values[node]=wvalue
        elseif length(node)==maxlen
            game_values[node]=0
        else
            children=get_children(payoffs, node)
            vals = [game_values[c] for c in children]
            if mod(len, 2)==1
                # player 2 to move: get minimum of children values
                game_values[node] = minimum(vals)
            else
                # player 1 to move: get maximum of children values
                game_values[node] = maximum(vals)
            end
        end
    end
end

In [20]:
game_values["%ZU7C`:#G5b"]

1.0

In [21]:
game_values["%ZU7C`:#G5"]

1.0

In [22]:
[nd => game_values[nd] for nd in keys(game_values) if length(nd)==10 && nd[1:9]=="%ZU7C`:#G"]

4-element Vector{Pair{String, Float64}}:
 "%ZU7C`:#G%" => 1.0
 "%ZU7C`:#G+" => 1.0
 "%ZU7C`:#G5" => 1.0
 "%ZU7C`:#G)" => 1.0

In [23]:
game_values["%ZU7C`:#G"]

1.0

In [24]:
game_values[node00]

1.0

In [25]:
# check length of best resistance
best_res_length = Dict{String,Int64}()
val = game_values[node00]
if val > 0
    resister = 1
else
    resister = 0 # player 2
end

1

In [26]:
for iter in 1:100
    node = node00
    val = game_values[node00]
    flag = true
    while flag
        cands = [c for c in keys(game_values) if length(c)==(length(node)+1) && c[1:length(node)]==node && game_values[c]==val]
        if length(cands) > 0
            node = rand(cands)
        else
            flag = false
        end
    end
    best_res_length[node] = length(node)-minlen
end
best_res_length

Dict{String, Int64} with 33 entries:
  "%ZU7C`:#L)c" => 3
  "%ZU7C`:#;5b" => 3
  "%ZU7C`:#;%c" => 3
  "%ZU7C`:#aEL" => 3
  "%ZU7C`:#L%b" => 3
  "%ZU7C`:#L+b" => 3
  "%ZU7C`:#;+b" => 3
  "%ZU7C`:#;+c" => 3
  "%ZU7C`:#;)c" => 3
  "%ZU7C`:#;)a" => 3
  "%ZU7C`:#L)a" => 3
  "%ZU7C`:#;5a" => 3
  "%ZU7C`:#bE;" => 3
  "%ZU7C`:#G)c" => 3
  "%ZU7C`:#G5b" => 3
  "%ZU7C`:#L3b" => 3
  "%ZU7C`:#G+b" => 3
  "%ZU7C`:#G%a" => 3
  "%ZU7C`:#;5c" => 3
  "%ZU7C`:#aE;" => 3
  "%ZU7C`:#;)b" => 3
  "%ZU7C`:#G+c" => 3
  "%ZU7C`:#G5a" => 3
  "%ZU7C`:#cEL" => 3
  "%ZU7C`:#;%b" => 3
  ⋮             => ⋮

In [27]:
# Range of BRs
lens_br = [length(k) for k in keys(best_res_length)]
minlen_br = minlen
maxlen_br = maximum(lens_br)

11

In [28]:
for len in maxlen_br:-1:minlen_br
    nodes = unique([node[1:len] for node in keys(best_res_length) if length(node) >= len])
    for node in nodes
        if haskey(best_res_length, node)
            # do nothing
        else
            children = [c for c in keys(best_res_length) if length(c) == (length(node)+1) && c[1:length(node)]==node]
            brs = [best_res_length[c] for c in children]
            if mod(length(node), 2)==resister
                # current node is resister to play
                best_res_length[node] = maximum(brs)
            else
                best_res_length[node] = minimum(brs)                
            end
        end
    end
end

In [29]:
# node = node00
# val = game_values[node00]
# flag = true
# while flag
#     cands = [c for c in nodes_by_length[length(node)-minlen+2] if c[1:length(node)]==node && game_values[c]==val]
#     if length(cands) > 0
#         node = rand(cands)
#     else
#         flag = false
#     end
# end
# display_path(payoffs, tree, node)

In [30]:
[nd => best_res_length[nd] for nd in keys(best_res_length) if length(nd)==1]

Any[]

In [31]:
best_res_length[node00]

3

In [32]:
game_values[node00]

1.0

### automate finding mate-in-2 problems

In [33]:
n_mc = 10000
beta = 20.0
for iter in 1:2
    game = ""
    #tree = create_tree(game)
    flag = true
    while flag
        ## MCTS turn
        tree = create_tree(game)
        tree = grow_tree(payoffs, tree, node=game, iters=n_mc)
        game = sample_next_node2(payoffs, tree, game, beta)
#         children = get_children(payoffs, game)
#         cands = Array{String}(undef, length(children))
#         nc = 0
#         # eliminate mates in 1, if possible
#         for c in children
#             value, br = mate_checker(payoffs, string(c), nits_tree = 1, verbose=false)
#             if (value == -1.0 && mod(length(c), 2)==1) || (value == 1.0 && mod(length(c), 2)==0)
#             else
#                 nc += 1
#                 cands[nc] = c
#             end
#         end
#         cands = cands[1:nc]
#         if length(cands) > 0
#             game = rand(cands)
#         else
#             println("Unavoidable mate")
#             display_node(payoffs, create_tree(game), game)
#             game = rand(children)
#         end
        if length(get_children(payoffs, game)) < 20
            print("Checking mate... ")
            #display_node(payoffs, tree, game)
            @time value, br = mate_checker(payoffs, game, nits_tree = 3, verbose=false)
            if abs(value)==1.0 && br == 3
                flag = false
                println("----MATE-IN-2 FOUND---")
                value, br = mate_checker(payoffs, game, nits_tree = 3, verbose=true)
                #print("value:")
                #print(value)
                #print(" best resist:")
                #println(br)
            end
        end
        if length(get_children(payoffs, game))==0
            flag = false
        end
    end
end

Checking mate...   0.068740 seconds (852.63 k allocations: 26.985 MiB, 12.41% compilation time)
Checking mate...   0.257576 seconds (2.86 M allocations: 108.201 MiB, 6.17% gc time)
Checking mate...   0.000282 seconds (3.53 k allocations: 135.711 KiB)
Checking mate...   0.447069 seconds (6.32 M allocations: 200.221 MiB, 7.60% gc time)
Checking mate...   0.815605 seconds (10.86 M allocations: 350.224 MiB, 8.70% gc time)
Checking mate...   0.000258 seconds (3.53 k allocations: 135.711 KiB)


## Mate in 3 analysis

In [ ]:
node0 = "c9\\"
display_node(payoffs, create_tree(node0), node0)
children1 = get_children(payoffs, node0);
gvalues = Dict(node00 => 0.0)

mi3 = ""
ord1 = sortperm(rand(length(children2)));
for node1 in children1[ord1]
    children2 = get_children(payoffs, node1)
    flag = true
    ord2 = sortperm(rand(length(children2)))
    ind = 1
    while flag
        node2 = children2[ord2[ind]]
        @time val, br = mate_checker(payoffs, node2, nits_tree=3, verbose=false)
        display_node(payoffs, create_tree(node2), node2)
        println(val)
        gvalues[node2] = val
        ind += 1
        if val >= 0.0
            # P1 has an escape
            flag = false
        end
        if ind > length(children2)
            println("^^^ MATE IN 3 ^^^")
            mi3 = string(mi3, "...", node1)
            flag = false
        end
    end
end